In [ ]:
#pytorch utility imports
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets

In [ ]:
#neural net imports
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [ ]:
#import libraries related to data Analysis and Visualisation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import math
%matplotlib inline

In [ ]:
#Check for CUDA

print(torch.cuda.is_available())
print(torch.backends.cudnn.enabled)

In [ ]:
# load the train and test data into python dataframe
input_folder_path = "input/"
train_df = pd.read_csv(input_folder_path+"train.csv")
test_df = pd.read_csv(input_folder_path+"test.csv")

In [ ]:
# extract trainimages , train lables and test images
train_labels = train_df['label'].values
train_images = (train_df.iloc[:,1:].values).astype('float32')
test_images = (test_df.iloc[:,:].values).astype('float32')

In [ ]:
#Training and Validation Split
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels,
                                                                     stratify=train_labels, random_state=123,
                                                                     test_size=0.20)

In [ ]:
train_images = train_images.reshape(train_images.shape[0], 28, 28)
val_images = val_images.reshape(val_images.shape[0], 28, 28)
test_images = test_images.reshape(test_images.shape[0], 28, 28)

In [ ]:
plt.imshow(train_images[10].squeeze(), cmap=plt.get_cmap('gray'))
plt.title(train_labels[10])

In [ ]:
# convert train_images array to tensor
train_images_tensor = torch.tensor(train_images)/255.0
train_labels_tensor = torch.tensor(train_labels)

# load TensorDataset
train_tensor = TensorDataset(train_images_tensor, train_labels_tensor)

In [ ]:
# conversion for validation set
val_images_tensor = torch.tensor(val_images)/255.0
val_labels_tensor = torch.tensor(val_labels)

# load TensorDataset for validation data
val_tensor = TensorDataset(val_images_tensor, val_labels_tensor)

In [ ]:
#test set tensor dataset
test_images_tensor = torch.tensor(test_images)/255.0

In [ ]:
# DataLoader objects
train_loader = DataLoader(train_tensor, batch_size=16, shuffle=True)
val_loader = DataLoader(val_tensor, batch_size=16, shuffle=True)
test_loader = DataLoader(test_images_tensor, batch_size=16,shuffle=False)

In [ ]:
# Linear model - size of input features, size of output features as parameters
linear_model = nn.Linear(28*28,10)

In [ ]:
#computes the cross entropy loss between input and target
criterion = torch.nn.CrossEntropyLoss() # computes softmax and then the cross entropy

In [ ]:
# Optimizer to implement stochastic gradient descent Algorithm
optimizer = torch.optim.SGD(linear_model.parameters(), 0.001)

In [ ]:
iter = 0
accuracy=0
for epoch in range(int(16)):
    print("epoch: {}. Accuracy: {}.".format(epoch, accuracy))
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28 * 28))
        labels = Variable(labels)
        
        # below method call is made to set the gradients of all optimized torch Tensors to zero.
        optimizer.zero_grad()
        
        #predict the output
        outputs = linear_model(images)
        
        #derive the loss
        loss = criterion(outputs, labels)
        
        # loss.backward method called to compute the gradients
        loss.backward()
        
        # below method called to perform single optimization step - i.e. to update the parameters
        optimizer.step()
        
        iter+=1
        if iter%500==0:
            # calculate Accuracy
            correct = 0
            total = 0
            for images, labels in val_loader:
                images = Variable(images.view(-1, 28*28))
                outputs = linear_model(images)
                _, predicted = torch.max(outputs.data, 1)
                total+= labels.size(0)
                # for gpu, bring the predicted and labels back to cpu fro python operations to work
                correct+= (predicted == labels).sum()
            accuracy = 100 * correct/total
            print("Iteration: {}. Loss: {}. Accuracy: {}.".format(iter, loss.item(), accuracy))